In [1]:
# Kaggle: https://www.kaggle.com/competitions/llm-classification-finetuning/
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
train=pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
test=pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')

In [3]:
train.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [4]:
test.head()

,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p..."


In [5]:
train.drop(columns=['model_a','model_b','id'], inplace=True)
train.head()

,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [6]:
train.tail()

,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
57472,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0
57473,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0
57474,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0
57475,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0
57476,"[""three kids eat three apples in three days, h...","[""27 apples""]","[""If three kids eat three apples in three days...",1,0,0


In [7]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57477 entries, 0 to 57476
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   prompt          57477 non-null  object
 1   response_a      57477 non-null  object
 2   response_b      57477 non-null  object
 3   winner_model_a  57477 non-null  int64 
 4   winner_model_b  57477 non-null  int64 
 5   winner_tie      57477 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 2.6+ MB


In [8]:
def clean_text(text):
    if pd.isnull(text):
        return text
    return (
        str(text)
        .encode('utf-8')         # Encode to bytes
        .decode('unicode_escape')# Decode unicode like \u03c0
        .replace('\n', ' ')      # Replace newlines with space
        .replace('\r', ' ')      # Handle carriage returns
        .replace('\"', '"')      # Fix quotes
        .replace("\'", "'")      # Fix apostrophes
        .replace('"', '')     # remove all double quotes
        .replace('[', '')     # remove all double quotes
        .replace(']', '')     # remove all double quotes
        .strip()                 # Trim leading/trailing whitespace
    )

# Apply cleaning to all text columns
for col in ['prompt', 'response_a', 'response_b']:
    train[col] = train[col].apply(clean_text)
    test[col] = test[col].apply(clean_text)    

/tmp/ipykernel_13/1314843304.py:7: DeprecationWarning: invalid escape sequence '\/'
  .decode('unicode_escape')# Decode unicode like \u03c0


In [9]:
train.tail()

,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
57472,A simple mnemonic for π: How I wish I could en...,"Sure, let's break it down: 1. How has 3 lette...",Here is how that mnemonic represents the digit...,1,0,0
57473,"In python, implement a naive Bayes with gaussi...",Here is an implementation of a naive Bayes cla...,Sure! Here's an implementation of a naive Baye...,1,0,0
57474,"is it unethical to work on building weapons?,a...",Working on weapons technology raises some ethi...,It depends on the context. Weapons can be used...,1,0,0
57475,"If a bait contains 0,0025% bromadiolon then ho...",Bromadiolone is a rodenticide which is most of...,"As an AI language model, I do not promote or c...",0,1,0
57476,"three kids eat three apples in three days, how...",27 apples,"If three kids eat three apples in three days, ...",1,0,0


In [10]:
train.head()

,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0
1,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0
2,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,0,0,1
3,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,1,0,0
4,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,0,1,0


In [11]:
test.head()

,id,prompt,response_a,response_b
0,136060,"I have three oranges today, I ate an orange ye...",You have two oranges today.,You still have three oranges. Eating an orange...
1,211333,You are a mediator in a heated political debat...,Thank you for sharing the details of the situa...,Mr Reddy and Ms Blue both have valid points in...
2,1233961,How to initialize the classification head when...,When you want to initialize the classification...,To initialize the classification head when per...


In [12]:
max_words = train['response_a'].apply(lambda x: len(str(x).split())).max()
print("Maximum number of words in any row:", max_words)

Maximum number of words in any row: 9150


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack  

vectorizer = TfidfVectorizer(max_features=10000)   

# Combine all text columns into one series for fitting
combined_text_train = train['prompt'] + ' ' + train['response_a'] + ' ' + train['response_b']
combined_text_test = test['prompt'] + ' ' + test['response_a'] + ' ' + test['response_b']

# Fit and transform
X_train_combined = vectorizer.fit_transform(combined_text_train)
X_test_combined = vectorizer.transform(combined_text_test)

# These are your final feature matrices
final_train_features = X_train_combined
final_test_features = X_test_combined

In [14]:
from lightgbm import LGBMClassifier, early_stopping
from sklearn.metrics import log_loss
from sklearn.model_selection import cross_val_predict, StratifiedKFold


# Map the labels to numerical values (0: winner_model_a, 1: winner_model_b, 2: winner_tie)
y = train[['winner_model_a', 'winner_model_b', 'winner_tie']].idxmax(axis=1).map({'winner_model_a': 0, 'winner_model_b': 1, 'winner_tie': 2})

X = final_train_features

# Set up cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation

fold = 0
train_losses = []
val_losses = []

for train_idx, val_idx in cv.split(X, y):
    fold += 1
    print(f"\n🔁 Fold {fold}")

    X_train, X_val = X[train_idx], X[val_idx]  
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # Initialize model
    model = LGBMClassifier(
        objective='multiclass',
        num_class=3,
        learning_rate=0.05,  # Lower learning rate to reduce overfitting
        n_estimators=2000,    # Number of boosting rounds
        random_state=42,
        max_depth=-1,         # Limit the depth of trees
        min_child_samples=20,  # Minimum samples per leaf
        min_child_weight=0.001,  # Minimum weight per leaf
        lambda_l1=0.75,       # L1 regularization
        lambda_l2=0.75,       # L2 regularization
        feature_fraction=0.8,  # Use a fraction of features
        bagging_fraction=0.8,  # Use a fraction of data
        bagging_freq=5,      # Perform bagging every 5 rounds
        early_stopping_rounds=30,  # Early stopping if no improvement
        eval_metric='multi_logloss'
    )
    
    # Train with logging
    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        eval_names=['train', 'valid'],
        eval_metric='multi_logloss',
    )

    # Predict probabilities
    train_probs = model.predict_proba(X_train)
    val_probs = model.predict_proba(X_val)

    # Compute log loss
    train_logloss = log_loss(y_train, train_probs)
    val_logloss = log_loss(y_val, val_probs)

    print(f"📊 Fold {fold} Log Loss - Train: {train_logloss:.4f}, Validation: {val_logloss:.4f}")
    train_losses.append(train_logloss)
    val_losses.append(val_logloss)

# Summary
print("\n✅ Cross-validated Results")
print(f"Mean Training Log Loss: {np.mean(train_losses):.4f}")
print(f"Mean Validation Log Loss: {np.mean(val_losses):.4f}")



🔁 Fold 1
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.75, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.75
[LightGBM] [Warning] early_stopping_round is set=30, early_stopping_rounds=30 will be ignored. Current value: early_stopping_round=30
[LightGBM] [Warning] lambda_l1 is set=0.75, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.75
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignor

In [15]:
test_probs = model.predict_proba(final_test_features)  # test_features is the feature matrix for your test set

# Predict the class labels on the test data
test_labels = model.predict(final_test_features)

# Display the predicted probabilities (for each class)
print("Test Probabilities:")
print(test_probs)

# Display the predicted class labels (0, 1, or 2)
print("Test Class Labels:")
print(test_labels)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.75, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.75
[LightGBM] [Warning] lambda_l1 is set=0.75, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.75
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.75, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7

In [16]:
prob_df = pd.DataFrame(test_probs, columns=['prompt', 'response_a', 'response_b'])
		
output = pd.concat([test['id'].reset_index(drop=True), prob_df], axis=1)

output.head()

,id,prompt,response_a,response_b
0,136060,0.278866,0.289883,0.431251
1,211333,0.404607,0.323820,0.271574
2,1233961,0.346599,0.386024,0.267377


In [17]:
output.to_csv('submission.csv', index=False)